# BayPass preparation

In [25]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [26]:
# Using maf_LR537144.csv as it's one of the smallest files we can use
biallelic_fish_maf=pd.read_csv(r"maf_LR537144.csv", sep=",")
biallelic_fish_maf

,chr,pos,index,ref_n,A,C,G,T,A.1,C.1,...,G.20,T.20,A.21,C.21,G.21,T.21,A.22,C.22,G.22,T.22
0,LR537144.1,10000025,6430776,C,0,67,0,92,0,75,...,0,7,0,56,0,11,0,35,0,30
1,LR537144.1,10000130,6430850,G,0,98,62,0,0,48,...,56,0,0,15,62,0,0,23,40,0
2,LR537144.1,10000201,6430923,T,0,15,0,152,0,27,...,0,56,0,17,0,77,0,17,0,55
3,LR537144.1,10000306,6431037,G,0,0,132,0,0,0,...,77,0,0,3,65,0,0,0,67,0
4,LR537144.1,10000606,6431194,T,0,0,0,122,0,0,...,0,75,0,0,0,77,0,0,0,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160545,LR537144.1,9999619,413689,A,41,0,142,0,48,0,...,41,0,42,0,32,0,32,0,37,0
160546,LR537144.1,9999796,413868,T,0,114,0,33,0,59,...,0,13,0,25,0,53,0,37,0,25
160547,LR537144.1,9999827,413895,a,131,0,11,0,91,0,...,0,0,92,0,0,0,61,0,2,0
160548,LR537144.1,9999900,413963,G,73,0,72,0,50,0,...,48,0,17,0,64,0,34,0,44,0


In [27]:
def BayPass_genotyping_file(infile, colnames):
    print("Starting BayPass genotyping file processing")
    
    # Convert input to numpy array for faster processing
    data = infile.iloc[:, 4:].values
    
    num_rows, num_cols = data.shape

    # Ensure the number of columns is divisible by 4
    if num_cols % 4 != 0:
        raise ValueError(f"Number of genotype columns ({num_cols}) is not divisible by 4.")
    
    # Reshape data to group every 4 columns
    data_reshaped = data.reshape(num_rows, -1, 4)
    num_groups = data_reshaped.shape[1]

    print(f"Input shape: {data.shape}, Reshaped: {data_reshaped.shape}")
    
    # Find non-zero indices for each row and group
    non_zero_mask = data_reshaped != 0
    
    # Initialize result array
    result = np.zeros((num_rows, num_groups * 2), dtype=data.dtype)
        
    for i in tqdm(range(num_rows), desc="Processing rows", unit="row"):
        try:
            row_data = data_reshaped[i]
            non_zero_indices = np.argwhere(row_data != 0)
            first_non_zero = non_zero_indices[0]
            second_non_zero = non_zero_indices[1]
            result[i, ::2] = row_data[first_non_zero[0], first_non_zero[1]]
            result[i, 1::2] = row_data[second_non_zero[0], second_non_zero[1]]
        except Exception as e:
            print(f"Error processing row {i}: {str(e)}")
    
    return pd.DataFrame(result, columns=colnames)


# Usage remains the same
column_names = ['wGRE_9_1', 'wGRE_9_2', 'wGRE_13_1', 'wGRE_13_2',
                'wSPA_5_1', 'wSPA_5_2', 'fSPA_3_1', 'fSPA_3_2',
                'fFRA_1_1', 'fFRA_1_2', 'fGRE_10_1', 'fGRE_10_2',
                'wTUR_14_1', 'wTUR_14_2', 'wSPA_4_1', 'wSPA_4_2',
                'wITA_8_1', 'wITA_8_2', 'fSPA_2_1', 'fSPA_2_2',
                'fGRE_9_1', 'fGRE_9_2', 'fGRE_8_1', 'fGRE_8_2',
                'fCRO_5_1', 'fCRO_5_2', 'wITA_7_1', 'wITA_7_2',
                'wGRE_12_1', 'wGRE_12_2', 'wGRE_11_1', 'wGRE_11_2',
                'wGRE_10_1', 'wGRE_10_2', 'fITA_4_1', 'fITA_4_2',
                'fGRE_6_1', 'fGRE_6_2', 'fGRE_7_1', 'fGRE_7_2']


In [28]:
# Usage
try:
    fish_maf_baypass = BayPass_genotyping_file(biallelic_fish_maf, column_names)
except Exception as e:
    print(f"Error in main execution: {str(e)}")

Starting BayPass genotyping file processing
Input shape: (160550, 92), Reshaped: (160550, 23, 4)


Processing rows: 100%|██████████| 160550/160550 [00:05<00:00, 30319.98row/s]


In [29]:
fish_maf_baypass

,wGRE_9_1,wGRE_9_2,wGRE_13_1,wGRE_13_2,wTUR_15_1,wTUR_15_2,fISR_11_1,fISR_11_2,wSPA_5_1,wSPA_5_2,...,fEGY_12_1,fEGY_12_2,wGRE_10_1,wGRE_10_2,fITA_4_1,fITA_4_2,fGRE_6_1,fGRE_6_2,fGRE_7_1,fGRE_7_2
0,67,92,67,92,67,92,67,92,67,92,...,67,92,67,92,67,92,67,92,67,92
1,98,62,98,62,98,62,98,62,98,62,...,98,62,98,62,98,62,98,62,98,62
2,15,152,15,152,15,152,15,152,15,152,...,15,152,15,152,15,152,15,152,15,152
3,132,114,132,114,132,114,132,114,132,114,...,132,114,132,114,132,114,132,114,132,114
4,122,152,122,152,122,152,122,152,122,152,...,122,152,122,152,122,152,122,152,122,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160545,41,142,41,142,41,142,41,142,41,142,...,41,142,41,142,41,142,41,142,41,142
160546,114,33,114,33,114,33,114,33,114,33,...,114,33,114,33,114,33,114,33,114,33
160547,131,11,131,11,131,11,131,11,131,11,...,131,11,131,11,131,11,131,11,131,11
160548,73,72,73,72,73,72,73,72,73,72,...,73,72,73,72,73,72,73,72,73,72


In [30]:
# BayPass analysis
# save without header and index, with space separation
fish_maf_baypass.to_csv(r"CHROMOSOME_NAME.genotype", sep=' ', index=False, header=False)

In [75]:
# TODO -- Change the filepaths for the shell commands

# ============== prepare the OTHER data files - inputs for the BayPass software ============== #

# These commands are for the Dlabrax species, we will adjust the names for Gilthead Seabream data

# the POOL (haploid) size file [require for Pool-Seq data]
# 2x the number of pooled individuals of each population

# covariate file - 0: farmed, 1: wild (-efile option - standard covariate model)

# contrast file - -1: farmed, 1: wild

# STANDARD COVARIATE MODEL with Importance Sampling approximation
# for the mac
./i_baypass -gfile ../seabass_47K_SNPs/47K_seabass_BayPass.genotype -efile ../seabass_47K_SNPs/47K_seabass_BayPass.covariate -contrastfile ../seabass_47K_SNPs/47K_seabass_BayPass.contrast -poolsizefile ../seabass_47K_SNPs/47K_seabass_BayPass.poolsize -outprefix 47K_seabass_run1 -nthreads 8 -nval 10000 -burnin 10000
# for the server (from inside the "MedFish/files/European_seabass/BayPass/baypass_2.3/sources" folder)
# for Dlabrax
./g_baypass -gfile ../seabass_Dlabrax_chromosome/labrax_Dlabrax_maf.genotype -efile ../seabass_Dlabrax_chromosome/labrax_Dlabrax_maf.covariate -contrastfile ../seabass_Dlabrax_chromosome/labrax_Dlabrax_maf.contrast -poolsizefile ../seabass_Dlabrax_chromosome/labrax_Dlabrax_maf.poolsize -outprefix labrax_Dlabrax_maf_baypass -nthreads 8 -nval 10000 -burnin 10000
# for vgll3
./g_baypass -gfile ../seabass_vgll3_chromosome/labrax_vgll3_maf.genotype -efile ../seabass_vgll3_chromosome/labrax_vgll3_maf.covariate -contrastfile ../seabass_vgll3_chromosome/labrax_vgll3_maf.contrast -poolsizefile ../seabass_vgll3_chromosome/labrax_vgll3_maf.poolsize -outprefix labrax_vgll3_maf_baypass -nthreads 8 -nval 10000 -burnin 10000

#     i) 20 pilot runs of  500 iterations (to adjust proposal distributions)
#    ii) a burn-in period of 10000 iterations
#   iii) final MCMC sampling of 10000 parameter values sampled every  20 iterations (i.e., 200000 iterations)

SyntaxError: invalid decimal literal (188991931.py, line 12)